### 모듈 import 하기

In [226]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import re
import urllib.request


import nltk
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
okt = Okt()

from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 데이터 전처리
- 데이터셋 불러오기
- 원하는 데이터프레임으로 만들어주기

In [227]:
badcomm = pd.read_csv('./비속어데이터셋_전처리o.csv')
badcomm.sample(10)

,댓글,비속어여부
5474,저렇게 고의적으로 밀어붙여놓고 이제와서 합의라,0
1495,견찰이 수사 종결해버릴 권리가 있다,0
812,한국은 좆같은 황사가 날아오지만역살적으로 땅의 회복력이 올라감,1
1926,자연적으로 저런상이 있다,0
960,내말이그말임 결국 적게벌면 적게버는 대로 스트레스일거고 그들처럼 돈이 많아서 걱정 ...,0
2224,진짜임 구라임,0
5700,이래서 애새끼 함부로 싸지르는거 아니다,1
5243,예전에 원룸이사해서 트럭불렀더니 틀딱아재가왔음아재하는말이 예전에 자기가 소돼지 이런...,1
2160,샌드백은 쿠팡에서 로켓배송 17700원짜리,0
513,쟤 누가뽑았냐,0


In [228]:
badcomm2 = pd.read_csv('./hate_speech_binary_dataset.csv')
badcomm2.head(10)

,문장,혐오 여부
0,정말 재밌다 연기도 좋고 디카프리오 짱,1
1,심쿵심쿵 미치네요이수혁땜에 잠 못자겠어요ㅠ자꾸 아른거림ㅠ,1
2,하지만 이니후빨러들은 이런거 관심 하나도 없음,0
3,@착한아이임당 A4 용지 덮고?,0
4,"진짜 평점 믿으면 안된 다는 걸 현실로 보여주는 영화입니다. 네티즌,전문가 하나같이...",1
5,조원진이 비례대표나 경북에서 공천 못받고도 입다물면 친중질히던거 이해할게,0
6,후!!!!!!!!!!! 쩔었다...,1
7,오랜만에 휴가내서 조카들 데리고 보고나왔어요 !! 조카들이 너무 좋아해서 평점 10...,1
8,제목만 보고도 더러운 머리스탈 떠올랐다 이름보다 아 그더런새끼겠지? 이랬다니까,0
9,중국 공산당 보는것 같다,0


In [229]:
badcomm2['혐오 여부'].dtypes

dtype('int64')

In [230]:
badcomm2['혐오 여부'] = badcomm2['혐오 여부'].map(lambda x: 0 if x == 1 else 1) 

In [231]:
badcomm2.sample(10)

,문장,혐오 여부
9734,저기요변한개주기도아까우시면보지마시던가요왜여기서악플을다세요?이상하면마음속에닫고사는거지...,0
161947,완전 재밌어용ㅋㅋ 소유 오늘이 막방이네요 소유편 더 해줬음좋겠어요ㅜㅜㅜ,0
24325,진짜 숨죽여 보게 만드는 최고의 영화였다.,0
114523,"재미있게 봤습니다, 사과가..ㅋㅋㅋ",0
183868,근데 황교익 거르고 누구 말이 맞는거냐,1
182146,역시 소중아\n한국은 제거되야한다,1
90070,그럼 그렇지 ㅋㅋㅋㅋㅋㅋㅋ\n\n아빠덕 듬뿍보면서 역차별 거리면서\n\n감성팔이 오...,1
145891,우윳빛깔한희정,0
41805,흐하하 재밌다 제시카 알바는 왜이리 예뻐???,0
2772,무슨 이런 개반전이 있지? 역시 쿠엔틴 타란티노~~,0


In [232]:
import re

badcomm2['문장'] = badcomm2['문장'].map(lambda x: re.compile("^@\w+\s").sub("",str(x)))


In [233]:
badcomm2.sample(10)

,문장,혐오 여부
82903,세월이 아무리 지나도 잊혀지지 않을 작품,0
188084,ㄹ혜 시발년때문에 나라 잘 돌아간다,1
183536,뻥카 아니냐 진짜 있으면 임기때 정리했겠지 뭐하하러 싹 남겨놓노,1
137372,시발 드럽게 까다롭노\n\n걍 뒤져시발,1
122966,보지새끼들 대가리 수준이 저렇지 뭐 ㅋㅋ\n무식한새끼들 투성이네\n보지들한테 자유를...,1
141749,별로지만 류현경때문에.......,0
43636,손예진이뽀,1
51363,거짓말은 아니지. 자기 명의는 아니니까,1
99053,안녕하세요. 저는 이 영화속 삶과 아주 흡사한 삶을 살고 있는 사람입니다.하지만 이...,0
115444,90년대의 걸작중에 하나죠...,0


In [234]:
badcomm2.rename(columns = {'문장':'댓글', '혐오 여부':'비속어여부'},inplace=True)

In [235]:
badcomm2

,댓글,비속어여부
0,정말 재밌다 연기도 좋고 디카프리오 짱,0
1,심쿵심쿵 미치네요이수혁땜에 잠 못자겠어요ㅠ자꾸 아른거림ㅠ,0
2,하지만 이니후빨러들은 이런거 관심 하나도 없음,1
3,A4 용지 덮고?,1
4,"진짜 평점 믿으면 안된 다는 걸 현실로 보여주는 영화입니다. 네티즌,전문가 하나같이...",0
...,...,...
189995,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,0
189996,케석대 어깨 올라간거봐라 ㅋㅋ,1
189997,day and night\n\nround the clock\n\nwithout a ...,1
189998,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,0


In [236]:
print(badcomm.shape)
print(badcomm2.shape)

(6769, 2)
(190000, 2)


In [237]:
badcomment = pd.concat([badcomm,badcomm2])
print(badcomment.shape)

(196769, 2)


In [238]:
badcomment['댓글'].nunique(), badcomment['비속어여부'].nunique()

(192443, 2)

In [239]:
badcomment['비속어여부'].unique()

array([1, 0])

In [240]:
badcomment.drop_duplicates(subset=['댓글'], inplace=True)

In [241]:
print('총 샘플의 수 :',len(badcomment))

총 샘플의 수 : 192443


In [242]:
badcomment.dtypes

댓글       object
비속어여부     int64
dtype: object

In [243]:
print(badcomment.groupby('비속어여부').size().reset_index(name = 'count'))

   비속어여부   count
0      0  100861
1      1   91582


In [244]:
print(badcomment.isnull().sum())

댓글       0
비속어여부    0
dtype: int64


In [245]:
# badcomment.to_csv('비속어댓글총합_전처리o.csv')

In [246]:
# 이모티콘, 특수문자 제거

def preprocessing(sentence):
    sentence_split = sentence.split()
    sentence_split_result = []
    for word in sentence_split:
        if word[0] != '@' and word[:4] !='http':
            sentence_split_result.append(word)
    sentence_result = ' '.join(sentence_split_result)
    text_result = ''.join(re.compile('[가-힣|0-9| ]+').\
                    findall(sentence_result)).strip()
    return text_result

badcomment["댓글"] = badcomment['댓글'].apply(preprocessing)

In [247]:
badcomment

,댓글,비속어여부
0,좌배 까는건,1
1,집에 롱 패딩만 세 개다 10년 더 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
...,...,...
189995,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,0
189996,케석대 어깨 올라간거봐라,1
189997,,1
189998,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,0


In [249]:
badcomment["댓글"] = badcomment["댓글"].str.replace('^ +', "") # white space 데이터를 empty value로 변경
badcomment["댓글"].replace('', np.nan, inplace=True)
print(badcomment.isnull().sum())

댓글       1862
비속어여부       0
dtype: int64


In [250]:
badcomment.loc[badcomment.댓글.isnull()][:]

,댓글,비속어여부
44,NaN,0
147,NaN,0
281,NaN,0
301,NaN,1
378,NaN,1
...,...,...
189706,NaN,0
189760,NaN,1
189949,NaN,1
189972,NaN,0


In [251]:
badcomment = badcomment.dropna(how = 'any')
print(len(badcomment))

190581


### 토큰화 - 불용어 제거

In [29]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다','부터']

In [30]:
# 데이터셋 분리

text = badcomment['댓글'] # 시리즈 객체로 저장
score = badcomment['비속어여부']

### train, test 데이터 분리

In [31]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(text, score , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

152464 152464 38117 38117


In [32]:
train_x

41047                                               술처럼 묵히노
89959                                        능력자냐 김앤장도 무서워함
134276    대학 시절 중국어 수업에서 보게 된 영화인데 마지막 장면에서 주인공이 돌아가신 아버...
182854                              여주인공 존내이쁘다고생각했는데 제니퍼였군하
130331         재방하는거봤는데 볼만하던데어이없으려면봐요재미남 7점주고싶은데 별점이불쌍해서10점
                                ...                        
150210                               꼭 봐야할 영화 오래 기억 남을것 같아요
175854                                                멋진 영화
114586    과거 일본이 한 짓이나 중공군 새끼들이 한 짓이나 별 다를바 없고 중국 새끼들은 오...
172444    오해했다 눈물찍 콧물찍 장면이 언제나오나 은근 기대하고 봤다 그러나 이 영화는 한국...
37814                                            성소게이는 변종이냐
Name: 댓글, Length: 152464, dtype: object

In [33]:
# 형태소 분석기를 사용하여 토큰화를 하면서 불용어를 제거하여 X_train에 저장

X_train = []
for sentence in tqdm(train_x):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화, 동사 기본형으로 변경
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|██████████| 152464/152464 [06:15<00:00, 406.56it/s]


In [31]:
print(X_train[:3])

[['술', '처럼', '묵다'], ['능력자', '냐', '김앤장', '무섭다', '함'], ['대학', '시절', '중국어', '수업', '에서', '보다', '되다', '영화', '인데', '마지막', '장면', '에서', '주인공', '돌아가다', '아버지', '대신', '아버지', '께서', '만들다', '교과서', '로', '가르치다', '모습', '가장', '감동', '적', '이다']]


In [34]:
# test 데이터도 형태소 분석기를 사용하여 토큰화를 하면서 불용어를 제거하여 X_test에 저장

X_test = []
for sentence in tqdm(test_x):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화, 동사 기본형으로 변경
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 38117/38117 [01:22<00:00, 460.59it/s]


In [35]:
print(X_test[:3])

[['오랜', '만', '재밋는', '영', '화보', '네', '요'], ['간만', '프랑스', '영화', '기대', '이상', '재미', '역시', '실망', '스럽지', '않다'], ['얘', '아', '부탁', '핸드폰', '읽다', '때', '가독성', '좋다', '누가', '사진', '편집', '올려주다', '게시', '글', '내용', '하고', '아래', '올라오다', '게시', '글', '중', '4', '대강', '예산', '22조', '랑', '현재', '문재인', '정부', '에서', '예산', '200조', '에서', '500조', '로', '늘리다', '거', '정리']]


### 정수 인코딩

In [36]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train) #fit으로 tokenization 적용해주기
print(tokenizer.word_index)

{'보다': 1, '영화': 2, '있다': 3, '이다': 4, '되다': 5, '좋다': 6, '다': 7, '없다': 8, '아니다': 9, '같다': 10, '로': 11, '적': 12, '에서': 13, '새끼': 14, '재밌다': 15, '진짜': 16, '너무': 17, '정말': 18, '만': 19, '그': 20, '안': 21, '않다': 22, '고': 23, '나': 24, '저': 25, '때': 26, '최고': 27, '것': 28, '아': 29, '생각': 30, '사람': 31, '못': 32, '게': 33, '거': 34, '인': 35, '내': 36, '말': 37, '하고': 38, '나오다': 39, '그렇다': 40, '더': 41, '가다': 42, '왜': 43, '들다': 44, '네': 45, '감동': 46, '보고': 47, '중': 48, '연기': 49, '임': 50, '수': 51, '개': 52, '자다': 53, '만들다': 54, '점': 55, '지': 56, '재미있다': 57, '오다': 58, '까지': 59, '알다': 60, '먹다': 61, '이렇다': 62, '드라마': 63, '야': 64, '모르다': 65, '지금': 66, '뭐': 67, '많다': 68, '요': 69, '주다': 70, '받다': 71, '면': 72, '그냥': 73, '인데': 74, '존나': 75, '다시': 76, '니': 77, '평점': 78, '병신': 79, '한국': 80, '나다': 81, '하': 82, '라': 83, '사랑': 84, '일': 85, '싶다': 86, '씨발': 87, '하나': 88, '애': 89, '맞다': 90, '문': 91, '1': 92, '서': 93, '정도': 94, '10': 95, '지다': 96, '치다': 97, '함': 98, '난': 99, '배우': 100, '나라': 101, '해주다': 102, '놈': 103, '한테':

In [37]:
# 빈도수가 낮은 단어들은 자연어 처리에서 배제
# 등장 빈도수가 2회인 단어들이 이 데이터에서 얼만큼의 비중을 차지하는지 확인

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0  # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0  # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0  # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value 

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1 # 빈도수가 2보다 작은 단어 개수 + 1
        rare_freq = rare_freq + value # 빈도수가 2보다 작은 단어들의 빈도수 총합

        
print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


단어 집합(vocabulary)의 크기 : 56786
등장 빈도가 2번 이하인 희귀 단어의 수: 31955
단어 집합에서 희귀 단어의 비율: 56.27267284189765
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.2273820288499113


In [38]:
# 등장 빈도수가 2이하인 단어들의 수를 제외한 단어의 개수를 단어 집합의 최대 크기로 제한

# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1

# 등장 빈도가 1회 이하인 단어들은 자연어 처리에서 별로 중요하지 않을 듯하여 정수인코딩 할때 없애줄것

vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 24832


In [39]:
# 케라스 토크나이저의 인자로 넘겨주고 텍스트 시퀀스를 정수 시퀀스로 변환
# 높은 정수가 부여된 단어들은 등장 빈도수가 매우 낮다
# 빈도수가 낮은 단어들은 자연어 처리에서 배제

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [40]:
# 정수 인코딩이 잘 되었나?
print(X_train[:5])

[[1259, 161, 3025], [5344, 109, 16285, 280, 98], [1332, 485, 4513, 3288, 13, 1, 5, 2, 74, 106, 181, 13, 274, 703, 721, 1534, 721, 2790, 54, 1802, 11, 1966, 285, 222, 46, 12, 4], [208, 274, 3950, 277, 30, 3887, 4, 643, 82], [4427, 1, 138, 19, 2188, 1, 151, 330, 620, 2086, 23, 86, 323, 55, 769, 95, 55]]


In [39]:
print(train_y)

41047     1
89959     1
134276    0
182854    0
130331    0
         ..
150210    0
175854    0
114586    1
172444    0
37814     1
Name: 비속어여부, Length: 152464, dtype: int64


In [40]:
print(test_y)

118978    0
31703     0
12840     1
97096     1
65493     0
         ..
120673    0
112552    1
77746     0
91085     1
82515     1
Name: 비속어여부, Length: 38117, dtype: int64


In [41]:
# y를 array 형식으로 저장해주기

y_train = np.array(train_y)
y_test = np.array(test_y)

In [42]:
print(y_train)
print(y_test)

[1 1 0 ... 1 0 1]
[0 0 1 ... 0 1 1]


### 패딩으로 댓글 길이 맞추기

In [42]:
print('리뷰의 최대 길이 :',max(len(review) for review in train_x))
print('리뷰의 평균 길이 :',sum(map(len, train_x))/len(train_x))

# 최대길이가 요상한 댓글이 있음

리뷰의 최대 길이 : 1018
리뷰의 평균 길이 : 34.611750970720955


In [43]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [44]:
max_len = 40
below_threshold_len(max_len, X_train)

# 97 % 가 40 이하이기 때문에 최대글자수는 40으로 패딩해주는것이 좋을것같다.

전체 샘플 중 길이가 40 이하인 샘플의 비율: 97.22360688424808


In [45]:
# 모든 샘플의 길이를 40으로 맞춘다 - 이때 정수인코딩한 데이터 사용!
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [46]:
print(X_train)

[[    0     0     0 ...  1259   161  3025]
 [    0     0     0 ... 16285   280    98]
 [    0     0     0 ...    46    12     4]
 ...
 [    0     0     0 ...     3    79    14]
 [    0     0     0 ...   105   278    30]
 [    0     0     0 ...   224 16023   305]]


### LSTM으로 분류하기
#### 하이퍼파라미터
- 임베딩 벡터 차원 = 100
- 은닉 상태 크기 = 128
- 배치 크기 = 64
- 에포크 = 15

In [48]:
# 두 개의 선택지 중 하나를 예측하는 이진 분류 문제를 수행하는 모델    

from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint # 조기종료

# embedding_dim = 100
# hidden_units = 128

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid')) # 로지스틱 회귀를 사용해야 하기 때문에 sigmoid

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model_v0.0.18.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
# mc = ModelCheckpoint('best_model_v0.0.12.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) # 훈련을 모니터링 하기 위한 지표 metrics=acc

2022-05-09 06:56:08.601624: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-09 06:56:08.631084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-05-09 06:56:08.631248: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-05-09 06:56:08.632405: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-09 06:56:08.633632: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-05-09 06:56:08.633858: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-05-

In [49]:

history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15
1906/1906 [==============================] - ETA: 0s - loss: 0.2338 - acc: 0.9054
Epoch 00001: val_acc improved from -inf to 0.91772, saving model to best_model_v0.0.18.h5
1906/1906 [==============================] - 65s 34ms/step - loss: 0.2338 - acc: 0.9054 - val_loss: 0.2070 - val_acc: 0.9177
Epoch 2/15
1905/1906 [============================>.] - ETA: 0s - loss: 0.1941 - acc: 0.9236
Epoch 00002: val_acc improved from 0.91772 to 0.92024, saving model to best_model_v0.0.18.h5
1906/1906 [==============================] - 65s 34ms/step - loss: 0.1941 - acc: 0.9236 - val_loss: 0.2033 - val_acc: 0.9202
Epoch 3/15
1905/1906 [============================>.] - ETA: 0s - loss: 0.1828 - acc: 0.9296
Epoch 00003: val_acc improved from 0.92024 to 0.92201, saving model to best_model_v0.0.18.h5
1906/1906 [==============================] - 65s 34ms/step - loss: 0.1828 - acc: 0.9296 - val_loss: 0.1987 - val_acc: 0.9220
Epoch 4/15
1905/1906 [============================>.] - ETA: 0s - loss

In [47]:
import os
os.listdir()

['.ipynb_checkpoints',
 '비속어데이터셋_전처리o.csv',
 'swearWord_basecode_v0.1.4.ipynb',
 'hate_speech_binary_dataset.csv',
 'best_model_v0.0.18.h5']

In [252]:
from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
loaded_model = load_model('best_model_v0.0.18.h5')

In [52]:
# 모델 정확도 테스트
# loaded_model = load_model('best_model_v0.0.18.h5')
# loaded_model = tf.keras.models.load_model('best_model_v0.0.18.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1192/1192 [==============================] - 8s 7ms/step - loss: 0.1967 - acc: 0.9238

 테스트 정확도: 0.9238


### 모델 사용해서 댓글 비속어 포함여부 예측해보기

In [165]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', str(new_sentence))
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.9):
        print("{:.2f}% 확률로 비속어가 포함된 댓글입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 일반 댓글입니다.\n".format((1 - score) * 100))

In [166]:
sentiment_predict('초코하임 맛있어요')

11.90% 확률로 일반 댓글입니다.



In [167]:
sentiment_predict('쓰레기들은 휴지통으로')

19.77% 확률로 일반 댓글입니다.



In [168]:
sentiment_predict('미쳤나 진짜 시발')

99.52% 확률로 비속어가 포함된 댓글입니다.



In [169]:
sentiment_predict('진주네요...여기 동네에 잠시 살았는데...여기 주차 장난아닙니다.....그냥 욕만 나오죠...')

67.13% 확률로 일반 댓글입니다.



In [170]:
sentiment_predict('애새끼가 초딩도 아니고')

99.15% 확률로 비속어가 포함된 댓글입니다.



In [171]:
sentiment_predict('성형후 사진 원래 포샵 존나 하는거 아님? 실제로 보면 오른쪽같이 안생겼을듯')

99.42% 확률로 비속어가 포함된 댓글입니다.



In [172]:
sentiment_predict('내말이 ㅋㅋ 아직 세상물정을 잘 모르는것 같더라...사회생활이 적었나..')

52.03% 확률로 일반 댓글입니다.



In [173]:
sentiment_predict('집가서 마라탕 먹자')

13.39% 확률로 일반 댓글입니다.



## 크롤링 데이터

In [202]:
df

,id,comment,program,time,love,play,result
18,마미얌,내장이 많이 있네요,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:08:06,97,"4,409",22.86% 확률로 일반 댓글입니다.
7,미니,홍게네요 오늘,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:02:42,27,"4,332",55.93% 확률로 일반 댓글입니다.
4,반지하의제왕,감사합니당 ^.^,시크릿상품_엔공구_대표님_빅딜!!_단_하루의_기회🎁,2022-05-02_13:10:03,"33,026","6,784",75.15% 확률로 일반 댓글입니다.
0,버뮤다,안녕하세요~,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:01:48,4,"4,308",29.26% 확률로 일반 댓글입니다.
34,장미같은,내장도 많길~ㅎ,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:21:03,568,"4,537",18.98% 확률로 일반 댓글입니다.
13,버뮤다,3키로면 몇마리인가요?,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:04:52,49,"4,365",44.10% 확률로 일반 댓글입니다.
16,간지,많이 들어있네요,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:06:26,66,"4,393",25.65% 확률로 일반 댓글입니다.
35,대박이,내장 상당히 많아 보이네요,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:21:03,568,"4,537",36.66% 확률로 일반 댓글입니다.
2,사막여우,대기했지렁,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:01:48,4,"4,308",14.74% 확률로 일반 댓글입니다.
9,미니,홍게~,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:02:42,27,"4,332",24.96% 확률로 일반 댓글입니다.


In [254]:
df = pd.read_csv('../naver_crawling_0502까지.csv', index_col=0)

In [130]:
# def sentiment_predict(new_sentence):
#     new_sentence = re.compile('[^ A-Za-z0-9ㄱ-ㅣ가-힣]+').sub('', str(new_sentence))
#     new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화        
#     new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
#     encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
#     pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
#     score = float(load_model).predict(pad_new) # 예측
#     if(score > 0.3): # 1에 가까워질수록 욕의 범위가 넓어짐, 0일때 필터링 강도가 최대
#         return "{:.2f}% 확률로 비속어가 포함된 댓글입니다.".format(score * 100)
#     else:
#         return "{:.2f}% 확률로 일반 댓글입니다.".format((1 - score) * 100)

In [197]:

# def swear_predict(new_sentence):
#     new_sentence = re.compile('[^ 0-9ㄱ-ㅣ가-힣]+').sub('', str(new_sentence))
#     new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화        
#     new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
#     encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
#     pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
#     score = float(loaded_model.predict(pad_new)) # 예측
#     if(score > 0.9): # 1에 가까워질수록 필터링강도가 약해지고, 0에 가까울때 필터링 강도가 최대
#         return "{:.2f}% 확률로 비속어가 포함된 댓글입니다.".format(score * 100)
#     else:
#         return "{:.2f}% 확률로 일반 댓글입니다.".format((1 - score) * 100)

In [208]:
# df = df.sample(50)
# df['result'] = df['comment'].map(lambda x: swear_predict(x))

In [209]:
# df[['result', 'comment']]

,result,comment
38,70.41% 확률로 일반 댓글입니다.,사은품이 뭔가요??
0,93.43% 확률로 비속어가 포함된 댓글입니다.,마스크도
17,81.38% 확률로 일반 댓글입니다.,언니가 입으니 엄청 이쁜 정비사 가툼
46,90.24% 확률로 일반 댓글입니다.,리모컨 설치는 테이프인가요?
17,24.96% 확률로 일반 댓글입니다.,NaN
65,95.81% 확률로 일반 댓글입니다.,대박인데요!!!!!!!
8,18.42% 확률로 일반 댓글입니다.,잘들려요ㅎㅎ
11,90.42% 확률로 비속어가 포함된 댓글입니다.,사각썰기가 이중이라니 그래서 식감이 좋을 거 같아여
31,84.87% 확률로 일반 댓글입니다.,언니 핑크는 진짜이뿜
36,78.14% 확률로 일반 댓글입니다.,하트 좋아여?


### 전처리 후 공백 ㅡ> 일반댓글, new_sentence 에 없을 경우/ 불용어일 경우 ㅡ> 일반댓글  

In [253]:
okt = Okt()
tokenizer = Tokenizer()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다','부터']
max_len = 40
vocab_size = 24832
tokenizer = Tokenizer(vocab_size)


def slang_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', str(new_sentence))
    if new_sentence == "" or new_sentence.isspace():
        return "일반 댓글"
    else : 
        new_sentence = okt.morphs(new_sentence, stem=True)
        new_sentence = [word for word in new_sentence if not word in stopwords]
        if not new_sentence:
            return "일반 댓글"
        else:
            encoded = tokenizer.texts_to_sequences([new_sentence])
            pad_new = pad_sequences(encoded, maxlen = max_len)
            score = loaded_model.predict(pad_new)
            if(score > 0.9): 
                return "비속어 포함"
            else:
                return "일반 댓글"
            
        
#         return round(float(score), 2)
        
#     if(score > 0.4): # 1에 가까워질수록 필터링강도가 약해지고, 0에 가까울때 필터링 강도가 최대
#         return "{:.2f}% 확률로 비속어가 포함된 댓글입니다.".format(score * 100)
#     else:
#         return "{:.2f}% 확률로 일반 댓글입니다.".format((1 - score) * 100)

In [255]:
# df2 = df.sample(50)
df['result'] = df['comment'].map(lambda x: slang_predict(x))

df

,id,comment,program,time,love,play,result
0,버뮤다,안녕하세요~,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:01:48,4,"4,308",일반 댓글
1,사막여우,방가워요,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:01:48,4,"4,308",일반 댓글
2,사막여우,대기했지렁,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:01:48,4,"4,308",일반 댓글
3,틴틴,홍게 너무 먹고싶었어요 ㅠㅠ,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:01:48,4,"4,308",일반 댓글
4,틴틴,저번 방송때 구매 못해서ㅜㅜ,부담없고_푸짐한_구룡포_연지홍게_라이브_특가행사!,2022-05-02_10:01:48,4,"4,308",일반 댓글
...,...,...,...,...,...,...,...
124,반짝이는 별빛,귀여워,리콜_나염_스커트_타임특가_후딱_델꼬가세요,2022-05-02_13:40:00,"6,024","3,057",일반 댓글
125,달콩스,와우~~ㅋㅋ,리콜_나염_스커트_타임특가_후딱_델꼬가세요,2022-05-02_13:40:00,"6,024","3,057",일반 댓글
126,hjbaab,와~~~~~넥라인 너무 이쁘네요,리콜_나염_스커트_타임특가_후딱_델꼬가세요,2022-05-02_13:40:00,"6,024","3,057",일반 댓글
127,꽃봄,요고 한여름에도 입을수 있는 소재인가용???,리콜_나염_스커트_타임특가_후딱_델꼬가세요,2022-05-02_13:40:00,"6,024","3,057",일반 댓글


In [269]:
df[['result', 'comment']].sample(15)

,result,comment
27,일반 댓글,👋
84,일반 댓글,이쁘드아 카레용기 ㅋㅋ
26,일반 댓글,안끓여도 되니까 회사에서도 마실 수있어서 좋아요! (끓여주긴 너무나 귀찮은것)
88,일반 댓글,즐겁게 보고갑니당~ 두구두구두구두구
5,일반 댓글,식스센스급 ㅋ
25,일반 댓글,하이체어에도 사용가능하네요.올~~
25,일반 댓글,좋은 혜택 감사합니다(*☻-☻*)
3,일반 댓글,♥2022050228904951♥구매인증요 진심이거먹고주부습진이다나았어요.피부과가실...
25,일반 댓글,2022050228135551 / 카네이션
27,일반 댓글,네 그런거같아요^^


In [270]:
def swear_predict(new_sentence):
    new_sentence = re.compile('[^ ㄱ-ㅎㅏ-ㅣ가-힣]+').sub('', str(new_sentence))
    if new_sentence == "" or new_sentence.isspace():
        return "일반 댓글"
    else:
        new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
        new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
        if not new_sentence:
            return "일반 댓글"
        else:    
            encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
            pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
            score = float(loaded_model.predict(pad_new)) # 예측
            if(score > 0.9):
                print(new_sentence)
                print("{:.2f}% 확률로 비속어가 포함된 댓글입니다.\n".format(score * 100))
                #  return round(float(score), 3)
            else:
                print(new_sentence)
                print("{:.2f}% 확률로 일반 댓글입니다.\n".format((1 - score) * 100))

In [271]:
df['result2'] = df['comment'].map(lambda x: slang_predict(x))

In [276]:
df[['comment','result','result2']].sample(10)

,comment,result,result2
25,오,일반 댓글,일반 댓글
69,하이!,일반 댓글,일반 댓글
12,어딜 그렇게 내려가세요,일반 댓글,일반 댓글
27,♥2022050228904951♥구매인증요진심이거먹고주부습진이다나았어요.피부과가실필...,일반 댓글,일반 댓글
41,많아요 마스크벗음 부끄러웡~^^화장안해도편하궁,일반 댓글,일반 댓글
27,쿄쿄님 편한거 인증~~~!!,일반 댓글,일반 댓글
34,보통 컬러마스크는,일반 댓글,일반 댓글
71,꼭 보증서 없어면 픽스제품이 아니지요,일반 댓글,일반 댓글
31,언니 핑크는 진짜이뿜,일반 댓글,일반 댓글
84,며느리는 반찬하는 옷 이어야죠,일반 댓글,일반 댓글
